Mateusz Reczulski 236440 Analiza Danych


# Analiza absencji w miejscu pracy

## Opis problemu
Zbiór danych dotyczy absencji pracowników w miejscu pracy. Dane są zbierane z międzynarodowej firmy.
Absencja pracowników może wpływać na produktywność firmy, a przez to na jej wyniki finansowe.
Możliwe, że analiza tych danych pomoże zrozumieć, które czynniki najbardziej wpływają na absencję w pracy.

## Źródło danych
Zbiór danych, który będziemy analizować, pochodzi z UCI Machine Learning Repository
i jest dostępny pod adresem: https://archive.ics.uci.edu/dataset/445/absenteeism+at+work

Zawiera on informacje na temat absencji w pracy, takie jak powód absencji, miesiąc roku, dzień tygodnia,
sezony, transport, odległość od miejsca zamieszkania do pracy, obciążenie pracą na dzień, czas spędzony
w pracy, liczba dzieci, palacz czy niepalacz, spożycie alkoholu, liczba zwierząt domowych, wiek, stan zdrowia, nieobecność.


# Importowanie bibliotek

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Wczytanie danych

In [30]:
dir = 'absenteeism+at+work/Absenteeism_at_work.csv'
data = pd.read_csv(dir, sep=";")

## Opis danych

Najpierw przyjrzymy się naszym danym, sprawdzając pierwsze pięć wierszy za pomocą metody `.head()`.


In [31]:
data.head()

### Statystyki opisowe

Metoda `.describe()` dostarcza statystyk opisowych dla wszystkich kolumn numerycznych w naszym DataFrame.
Dla każdej kolumny otrzymamy następujące informacje:
- `count` - ilość rekordów
- `mean` - średnia wartość
- `std` - odchylenie standardowe
- `min` - minimalna wartość
- `25%`, `50%`, `75%` - odpowiednio pierwszy kwartyl, mediana (drugi kwartyl) oraz trzeci kwartyl
- `max` - maksymalna wartość


In [32]:
data.describe()

### Wizualizacje danych

Visualizacje danych są kluczowe do zrozumienia struktury naszych danych. Poniżej znajdują się dwa przykłady wizualizacji, które możemy wykorzystać.

#### Mapa ciepła (Heatmap)

Pierwsza to mapa ciepła korelacji, która pokazuje, jak nasze cechy są ze sobą powiązane. Na osiach mamy nazwy cech, a kolor każdej komórki reprezentuje siłę korelacji między cechami oznaczonymi na osiach.

In [33]:
correlation_data = data.drop('ID', axis=1).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm')
plt.title("Mapa ciepła korelacji cech")
plt.show()

#### Histogramy

Histogramy są używane do wyświetlania rozkładu jednowymiarowego zestawu obserwacji. Dla każdej kolumny numerycznej w naszych danych, wygenerujemy histogram, który pokaże nam, jak często pojawiają się różne zakresy wartości.

In [34]:
data.drop('ID', axis=1).hist(bins=30, figsize=(20,15))
plt.show()

## Przygotowanie danych

### Czyszczenie danych

Często w realnych zestawach danych pojawiają się brakujące wartości. Wykorzystamy metodę `.isnull().sum()` do sprawdzenia, czy w naszych danych są jakieś brakujące wartości.

In [35]:
# Sprawdzanie brakujących danych
data.isnull().sum()

W przypadku braku danych możemy zdecydować się na różne strategie, na przykład usunięcie tych rekordów,
wypełnienie ich średnią lub medianą, bądź zastosowanie bardziej zaawansowanych metod takich jak
imputacja wielokrotna.

W tym przypadku, zakładamy że nie ma brakujących danych. Jeśli jednak takowe wystąpiły,
poniżej znajduje się przykładowy kod, który wypełnia brakujące wartości medianą.

In [36]:
# Wypełnianie brakujących danych medianą
# data.fillna(data.median(), inplace=True)

### Wybór cech

Następnie dokonamy selekcji cech, które chcemy wykorzystać do modelowania. Zakładamy, że kolumna
"Cecha_do_Prognozowania" jest naszą zmienną docelową, a wszystkie inne kolumny numeryczne to nasze cechy.



Wykorzystamy algorytm Lasów Losowych do określenia ważności poszczególnych cech w naszym zbiorze danych.


In [37]:
features = data.drop('Age', axis=1)
target = data['Age']
from sklearn.ensemble import RandomForestRegressor

# Tworzymy model Lasu Losowego
forest = RandomForestRegressor(n_estimators=100, random_state=42)

# Zakładając, że features_scaled i target zostały wcześniej zdefiniowane
forest.fit(features_scaled, target)

# Pobieramy ważności cech
importances = forest.feature_importances_

# Tworzymy DataFrame dla lepszej wizualizacji
feature_importances = pd.DataFrame({
    'Feature': features.columns,  # Zakładając, że 'features' jest DataFrame
    'Importance': importances
})

# Sortujemy cechy według ważności
sorted_features = feature_importances.sort_values(by='Importance', ascending=False)

# Wizualizacja
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=sorted_features)
plt.title('Ważność cech według Lasów Losowych')
plt.show()


Na podstawie powyższej wizualizacji możemy wybrać cechy o największej ważności dla naszego problemu.

### Wybór cech przy użyciu metody SelectKBest

Używamy testu statystycznego f_classif do oceny cech.

In [38]:
from sklearn.feature_selection import SelectKBest, f_classif

# Zakładając, że chcemy wybrać 10 najlepszych cech
selector = SelectKBest(score_func=f_classif, k=10)

# Używamy selektora do oceny cech (z wyeliminowaniem 'ID')
features_without_id = features.drop('ID', axis=1)
best_features = selector.fit_transform(features_without_id, target)

# Tworzymy DataFrame dla lepszej wizualizacji
feature_scores = pd.DataFrame({
    'Feature': features_without_id.columns, 
    'Score': selector.scores_
})

# Sortujemy cechy według wyniku
sorted_scores = feature_scores.sort_values(by='Score', ascending=False)

# Wizualizacja
plt.figure(figsize=(12, 8))
sns.barplot(x='Score', y='Feature', data=sorted_scores)
plt.title('Wyniki cech według metody SelectKBest')
plt.show()

Na podstawie powyższej wizualizacji możemy wybrać cechy o najwyższym wyniku, co wskazuje na ich potencjalne znaczenie dla naszego problemu klasyfikacji.

### Rekursywna eliminacja cech (RFE)

Używamy klasyfikatora SVM jako estymatora do oceny ważności cech i rekursywnego ich eliminowania.

In [39]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

# Zakładając, że chcemy wybrać 10 najlepszych cech
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)

# Używamy RFE do oceny cech (z wyeliminowaniem 'ID')
features_without_id = features.drop('ID', axis=1)
selector = selector.fit(features_without_id, target)

# Tworzenie DataFrame dla lepszej wizualizacji
feature_ranking = pd.DataFrame({
    'Feature': features_without_id.columns, 
    'Ranking': selector.ranking_
})

# Sortujemy cechy według rankingu
sorted_ranking = feature_ranking.sort_values(by='Ranking')

# Wizualizacja
plt.figure(figsize=(12, 8))
sns.barplot(x='Ranking', y='Feature', data=sorted_ranking)
plt.title('Ranking cech według metody RFE')
plt.show()


Na podstawie powyższej wizualizacji możemy wybrać cechy o najniższym rankingu, co wskazuje na ich potencjalne znaczenie dla naszego problemu klasyfikacji. Im niższa wartość rankingu, tym ważniejsza jest cecha.

### Połączenie trzech metod selekcji cech



In [40]:
# Metoda Lasów Losowych
forest = RandomForestRegressor(n_estimators=100, random_state=42)
forest.fit(features_without_id, target)
importances_rf = forest.feature_importances_

# Metoda SelectKBest
selector_kbest = SelectKBest(score_func=f_classif, k=features_without_id.shape[1]) # wybieramy wszystkie cechy
best_features_kbest = selector_kbest.fit(features_without_id, target)
scores_kbest = selector_kbest.scores_

# Metoda RFE
estimator = SVC(kernel="linear")
selector_rfe = RFE(estimator, n_features_to_select=1) # ranking wszystkich cech
selector_rfe = selector_rfe.fit(features_without_id, target)
ranking_rfe = selector_rfe.ranking_

# Połączenie wyników
combined_scores = pd.DataFrame({
    'Feature': features_without_id.columns,
    'Importance_RF': importances_rf,
    'Score_KBest': scores_kbest,
    'Ranking_RFE': max(ranking_rfe) - ranking_rfe + 1, # odwracamy ranking, aby większa wartość oznaczała ważniejszą cechę
})

combined_scores['Combined_Score'] = combined_scores['Importance_RF'] + combined_scores['Score_KBest'] + combined_scores['Ranking_RFE']

# Sortujemy według skumulowanego wyniku
sorted_combined_scores = combined_scores.sort_values(by='Combined_Score', ascending=False)

# Wizualizacja
plt.figure(figsize=(12, 8))
sns.barplot(x='Combined_Score', y='Feature', data=sorted_combined_scores)
plt.title('Najważniejsze cechy według połączonych wyników trzech metod')
plt.show()


### Przekształcenia danych

Możemy również przekształcić nasze dane, na przykład przez normalizację. Wiele modeli działa lepiej,
gdy dane są znormalizowane. Poniżej znajduje się przykładowy kod, który wykonuje normalizację
za pomocą metody Min-Max.

In [41]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [42]:
# Import niezbędnych bibliotek i podział danych
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Podziel dane na cechy i etykiety (target)
X = data.drop('Absenteeism time in hours', axis=1)
y = data['Absenteeism time in hours']

# Podziel dane na zbiory treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Przekształcenie wybranych cech
# Wybierz wybrane cechy
selected_features = ['Weight', 'Service time', 'Distance from Residence to Work', 'Social drinker', 'Transportation expense']

# Przekształć dane tylko dla wybranych cech
scaler = StandardScaler()
X_train_selected = scaler.fit_transform(X_train[selected_features])
X_test_selected = scaler.transform(X_test[selected_features])



### Dyskretyzacja danych

In [43]:
# Dyskretyzacja kolumny "Distance from Residence to Work"
bins = [0, 10, 30, 50]  # zakresy
labels = ['blisko', 'średnio daleko', 'daleko']
X_train['Distance from Residence to Work'] = pd.cut(X_train['Distance from Residence to Work'], bins=bins, labels=labels, right=False)
X_test['Distance from Residence to Work'] = pd.cut(X_test['Distance from Residence to Work'], bins=bins, labels=labels, right=False)

# Wyświetl pierwsze 5 wierszy, aby zobaczyć efekt dyskretyzacji
X_train[['Distance from Residence to Work']].head()
